In [ ]:

import requests
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas
import seaborn as sns 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter 
import re
import string
import matplotlib.cm as cm
from matplotlib import rcParams
from prettytable import PrettyTable
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
import numpy as np 
import pandas as pd 
import re
import nltk
nltk.download("stopwords") 
from sklearn.model_selection import train_test_split

**read data**

In [ ]:
all_dialect_dataset=pd.read_csv("/content/drive/MyDrive/all_dialect_dataset.csv",lineterminator='\n').drop("id.1",axis=1)

In [ ]:
print(all_dialect_dataset.shape)
all_dialect_dataset.head(5)

In [ ]:
all_dialect_dataset["dialect"].nunique()

In [ ]:
np.sum(all_dialect_dataset.isnull().any(axis=1)) # there is no null values 

**Data preprocessing**

In [ ]:
Data['clean_tweet'] = Data.txt.str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE) #removes emojis

Data['clean_tweet'] = Data.clean_tweet.str.replace('@[_A-Za-z0-9]+', '') #removes handles

Data['clean_tweet'] = Data.clean_tweet.str.replace('RT', '') #removes rt

Data['clean_tweet'] = Data.clean_tweet.str.replace('#',' ') #removes hashtag symbol only

Data['clean_tweet'] = Data.clean_tweet.str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True) #removes links

Data['clean_tweet'] = Data.clean_tweet.str.replace('\d+', '') #removes numbers

Data['clean_tweet'] = Data.clean_tweet.str.replace('\n', ' ') #removes new line

Data['clean_tweet'] = Data.clean_tweet.str.replace('_', '') #removes underscore

Data['clean_tweet'] = Data.clean_tweet.str.replace('[^\w\s]','') #removes punctuation

In [ ]:
#normalizearabic
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
Data['clean_tweet'] = Data['clean_tweet'].apply(normalize_arabic)   

In [ ]:
#remove repeating characters
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
Data['clean_tweet'] = Data['clean_tweet'].apply(remove_repeating_char)   

In [ ]:
# remove Non Arabic characters
def removeNonArabicChar(text):
    return re.sub(r'[^0-9\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD.0-9]+', ' ', text)
Data['clean_tweet'] = Data['clean_tweet'].apply(removeNonArabicChar)   

In [ ]:
# remove Unnecessary Spaces
def removeUnnecessarySpaces(text):
    return re.sub(r'[\n\t\ ]+', ' ', text)
Data['clean_tweet'] = Data['clean_tweet'].apply(removeUnnecessarySpaces)

In [ ]:
Data.dialect.value_counts(normalize=True)*100 # imbalanced data problem

In [ ]:
# remove stop words
arb_stopwords = set(nltk.corpus.stopwords.words("arabic")) #arabic stopwords are not biult-in, so we find them by calling a set object

def delete_stop_words(mess):

    extra=["من","الله","اله","كى","رب","خلا","عدا","حاشا","عن","حتي","و","فى","انا","علي","الى","انت","الي","انتما","واله","مش","ان","او","اذا","انتم","وانا","انتن","هذا","هذه","هؤلاء","هم","هن","هى","هو"]
    stop_words = set(nltk.corpus.stopwords.words("arabic")+extra)
    
    return ' '.join(word for word in mess.split() if word not in stop_words and len(word)>1)
    
Data['clean_tweet'] = Data['clean_tweet'].apply(delete_stop_words)

In [ ]:
print(arb_stopwords)

In [ ]:
data = Data['clean_tweet'].astype(str)
target = Data['dialect'].astype('category')

# Split dataset into training set and test set before feature extraction
X_train, X_test, y_train, y_test = train_test_split(data,target, test_size=0.3,random_state=109, shuffle=True,stratify=target) 

In [ ]:
# FOR ML MODEL

In [ ]:
#feature extractin starts with counting the unique features
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#then transforms the features
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False)
X_train_tf = tf_transformer.fit_transform(X_train_counts)
X_train_tf.shape

In [ ]:
features = X_train_tf
labels = y_train
features.shape

In [ ]:
# for deep learning model

In [ ]:
max_length = 50
vocab_size = 100000
trunc_type = 'post'
padding_type = 'post'

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

In [ ]:
train_sequences = tokenizer.texts_to_sequences(X_train)
print(train_sequences[10])

In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating=trunc_type)
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

In [ ]:
validation_sequences = tokenizer.texts_to_sequences(X_test)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(target)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(y_train))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(y_test))
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)